**Step 1: Extract data to dataframe

In [1]:
import pandas as pd

# Load the Excel file to dataframe
excel_file = "E:\VCB\My notebooks\data DAE.xlsx"

df_fact = pd.read_excel(excel_file, sheet_name='fact')
df_d_scheme = pd.read_excel(excel_file, sheet_name='d_scheme')
df_d_voucher = pd.read_excel(excel_file, sheet_name='d_voucher')
df_d_merchant = pd.read_excel(excel_file, sheet_name='d_merchant')
df_d_user = pd.read_excel(excel_file, sheet_name='d_user')

**Step 2: Review data model

*Vấn đề 1: Bảng fact có 2 fields AVAILABLE_FROM, AVAILABLE_TO trùng lặp với 2 fields display_date_from, display_date_to tại bảng d_voucher 
-> Remove 2 fields này trên bảng fact

*Vấn đề 2: Fields VOUCHER_NAME của bảng fact lưu trữ thuộc tính của voucher_code 
-> Remove và chuyển sang lưu trữ tại bảng d_voucher

**Step 3: Sửa đổi data model và tạo bảng vật lý trên SQL Server theo file create_object.sql


**Step 4: Transform and Load data to SQL server

In [2]:
import sqlalchemy
# SQL Server connection details
server = 'DESKTOP-6C72I8E\SQLEXPRESS'
database = 'OneU'
driver = 'ODBC Driver 17 for SQL Server'

# Create the connection string using SQLAlchemy's format
connection_string = f"mssql+pyodbc://@{server}/{database}?driver={driver}&trusted_connection=yes"

# Create the SQLAlchemy engine
engine = sqlalchemy.create_engine(connection_string)
print("Connection to SQL Server successful!")

Connection to SQL Server successful!


- Bảng fact

In [5]:
# Bảng fact
df_fact_clean = df_fact.drop(['VOUCHER_NAME', 'AVAILABLE_FROM', 'AVAILABLE_TO'], axis=1)
df_fact_clean['CALENDAR_DIM_ID'] = pd.to_datetime(df_fact_clean['CALENDAR_DIM_ID'], format='%Y-%m-%dT%H:%M')
df_fact_clean = df_fact_clean.rename(columns={'TRANSACTION_ID': 'transaction_id'
                                         ,'SERIAL_NUMBER': 'serial_number'
                                         ,'VOUCHER_CODE': 'voucher_code'
                                         ,'MERCHANT_CODE': 'merchant_code'
                                         ,'ACTION': 'action'
                                         ,'CALENDAR_DIM_ID': 'calendar_dim_id'
                                         ,'USER_ID': 'user_id'
                                         })

fact_data_types = {
    'transaction_id': sqlalchemy.types.Integer(),
    'serial_number': sqlalchemy.types.VARCHAR(length=255),
    'voucher_code': sqlalchemy.types.VARCHAR(length=255),
    'merchant_code': sqlalchemy.types.VARCHAR(length=255),
    'action': sqlalchemy.types.VARCHAR(length=255),
    'calendar_dim_id': sqlalchemy.types.DATE(),
    'user_id': sqlalchemy.types.VARCHAR(length=255)
    }
#Load to database
df_fact_clean.to_sql(name='fact', con=engine, if_exists='append', index=False, dtype=fact_data_types, schema = 'voucher')
print("Load data to table fact done!")

Load data to table fact done!


- Bảng d_scheme

In [6]:
# Bảng d_scheme
df_d_scheme_clean = df_d_scheme.rename(columns={'VOUCHER_CODE': 'voucher_code'})
d_scheme_data_types = {
    'discount_type': sqlalchemy.types.VARCHAR(length=255),
    'discount_amount': sqlalchemy.types.DECIMAL(10,2),
    'discount_percent': sqlalchemy.types.DECIMAL(10,2),
    'description': sqlalchemy.types.VARCHAR(length=255),
    'voucher_code': sqlalchemy.types.VARCHAR(length=255)
}
#Load to database
df_d_scheme_clean.to_sql(name='d_scheme', con=engine, if_exists='append', index=False, dtype=d_scheme_data_types, schema = 'voucher')
print("Load data to table d_scheme done!")

Load data to table d_scheme done!


- Bảng d_voucher

In [7]:
# Bảng d_voucher
# Thực hiện phép LEFT JOIN để lấy thêm cột voucher_name từ df_fact
df_fact_rename = df_fact.rename(columns={'VOUCHER_CODE': 'voucher_code', 'VOUCHER_NAME': 'voucher_name'})
df_voucher_master = df_fact_rename[['voucher_code', 'voucher_name']].drop_duplicates()

df_d_voucher_merge = pd.merge(df_d_voucher, df_voucher_master, on='voucher_code', how='left')
df_d_voucher_clean = df_d_voucher_merge[['voucher_code','voucher_name','total_stock','display_date_from','display_date_to']]
# Change data types
df_d_voucher_clean['display_date_from'] = pd.to_datetime(df_d_voucher_clean['display_date_from'])
df_d_voucher_clean['display_date_to'] = pd.to_datetime(df_d_voucher_clean['display_date_to'])

d_voucher_data_types = {
    'voucher_code': sqlalchemy.types.VARCHAR(length=255),
    'voucher_name': sqlalchemy.types.NVARCHAR(length=255),
    'total_stock': sqlalchemy.types.Integer(),
    'display_date_from': sqlalchemy.types.DATETIME(),
    'display_date_to': sqlalchemy.types.DATETIME()
}
#Load to database
df_d_voucher_clean.to_sql(name='d_voucher', con=engine, if_exists='append', index=False, dtype=d_voucher_data_types, schema = 'voucher')
print("Load data to table d_voucher done!")

Load data to table d_voucher done!


- Bảng d_merchant

In [8]:
# Bảng d_merchant
df_d_merchant_null_filtered = df_d_merchant[df_d_merchant['MERCHANT_CODE'].isna() | df_d_merchant['category'].isna() | df_d_merchant['sub_category'].isna()]
print(df_d_merchant_null_filtered)
# Đánh giá các giá trị NaN thay thế bằng 'Khác'
df_d_merchant_filled = df_d_merchant.fillna('Khác')
df_d_merchant_clean = df_d_merchant_filled.rename(columns={'MERCHANT_CODE': 'merchant_code'})
# Tìm các giá trị bị trùng lặp trong cột 'merchant_code'
duplicate_merchant_codes = df_d_merchant_clean[df_d_merchant_clean.duplicated('merchant_code', keep=False)]

# Hiển thị các giá trị trùng lặp
print(duplicate_merchant_codes['merchant_code'].unique())

# Để thuận tiện trong việc tính toán các metric liên quan đến category, sub_category, cần xử lý các dòng có chứa các giá trị trùng lặp này
# --> Sau khi đánh giá về ý nghĩa và tần suất xuất hiện category, sub_category của các merchant_code trong bảng fact, em chọn phương pháp chọn giá trị đại diện category
# và sub_category cho các merchant_code
# Ví dụ đối với Merchant_code ='xXC'
# Tỷ lệ số giao dịch category = Tiện ích - Nhà mạng và sub_category= Viễn thông trên tổng số giao dịch tại bảng fact là 92.4% (65804/71201)
# --> Chọn category = Tiện ích - Nhà mạng và sub_category= Viễn thông làm đại diện cho merchant = 'xXC'
# Làm tương tự với 5 merchant_code còn lại ta thu được kết quả như sau:
# Xóa tất cả các dòng mà cột merchant_code bị duplicate
df_d_merchant_clean = df_d_merchant_clean[~df_d_merchant_clean['merchant_code'].duplicated(keep=False)]
# Tạo DataFrame với 6 dòng mới
new_rows = pd.DataFrame({
    'merchant_code': ['xXC', 'DOSALESFORCE23221RiK', '0316282840MISHI', 'DOSALESFORCE22152Jzp', 'VUAHANGHIEU', 'DOSALESFORCE23359nRU'],
    'category': ['Tiện ích - Nhà mạng', 'Sức khỏe - Làm đẹp', 'Nhà cửa - Đời sống', 'Vận tải - Giao vận', 'Thời trang và Phong cách sống', 'Sức khỏe - Làm đẹp'],
    'sub_category': ['Viễn thông', 'Mỹ phẩm - Chăm sóc da, tóc', 'Gia dụng', 'Dịch vụ giao hàng', 'Thời trang', 'Thực phẩm chức năng - CSSK']
})

# Thêm các dòng mới vào Dataframe
df_d_merchant_clean = pd.concat([df_d_merchant_clean, new_rows], ignore_index=True)


d_merchant_data_types = {
    'merchant_code': sqlalchemy.types.VARCHAR(length=255),
    'category': sqlalchemy.types.NVARCHAR(length=255),
    'sub_category': sqlalchemy.types.NVARCHAR(length=255)
}
#Load to database
df_d_merchant_clean.to_sql(name='d_merchant', con=engine, if_exists='append', index=False, dtype=d_merchant_data_types, schema = 'voucher')
print("Load data to table d_merchant done!")


           MERCHANT_CODE            category sub_category
23       0316282840MISHI                Khác          NaN
29                 VinID                 NaN          NaN
40             Innisfree                 NaN          NaN
57                   xXC             Mẹ & bé          NaN
66                   xXC  Thương mại điện tử          NaN
73  DOSALESFORCE24088ABs             Mẹ & bé          NaN
94  DOSALESFORCE24036I0u             Mẹ & bé          NaN
['xXC' 'DOSALESFORCE23221RiK' '0316282840MISHI' 'DOSALESFORCE22152Jzp'
 'VUAHANGHIEU' 'DOSALESFORCE23359nRU']
Load data to table d_merchant done!


- Bảng d_user

In [9]:
# Bảng d_user
# Làm sạch cột Gender
gender_values = df_d_user['gender'].unique()
print("Danh sách các giá trị gender:", gender_values)

rows_with_null_gender = len(df_d_user[df_d_user['gender'].isna()])
print("Số dòng có gender null:", rows_with_null_gender)

#Thay thế các giá trị null, nan, unavailable bằng giá trị UNIDENTIFIED
df_d_user['gender'] = df_d_user['gender'].replace([None, pd.NA, 'unavailable'], 'UNIDENTIFIED')

gender_values = df_d_user['gender'].unique()
print("Danh sách các giá trị gender sau chỉnh sửa:", gender_values)


Danh sách các giá trị gender: ['F' nan 'M' 'unavailable']
Số dòng có gender null: 6248
Danh sách các giá trị gender sau chỉnh sửa: ['F' 'UNIDENTIFIED' 'M']


In [10]:
# Làm sạch cột province_name
province_values = df_d_user['province_name'].unique()
print("Danh sách các giá trị province_name:", province_values)

# Làm sạch các giá trị cho chuẩn chung định dạng, các giá trị về Miền Bắc, Miền Trung, Miền Nam lấy các tỉnh lần lượt làm đại diện Hà Nội, Đà Nẵng, Hồ Chí Minh 
df_d_user['province_name'] = df_d_user['province_name'].replace(['HÀ NỘI', 'Hànoi', 'Miền Bắc'], 'Hà Nội')
df_d_user['province_name'] = df_d_user['province_name'].replace(['HCM','Miền Nam'], 'Hồ Chí Minh')
df_d_user['province_name'] = df_d_user['province_name'].replace(['ĐN','Miền Trung'], 'Đà Nẵng')

province_values = df_d_user['province_name'].unique()
print("Danh sách các giá trị province_name sau khi chỉnh sửa:", province_values)
df_d_user

Danh sách các giá trị province_name: ['Hà Nội' 'Miền Trung' 'Hồ Chí Minh' 'Miền Bắc' 'HÀ NỘI' 'UNIDENTIFIED'
 'HCM' 'ĐN' 'Miền Nam' 'Hànoi' 'Đà Nẵng']
Danh sách các giá trị province_name sau khi chỉnh sửa: ['Hà Nội' 'Đà Nẵng' 'Hồ Chí Minh' 'UNIDENTIFIED']


,gender,user_id,province_name,age
0,F,76843,Hà Nội,O55
1,UNIDENTIFIED,13346d5,Đà Nẵng,25_34
2,UNIDENTIFIED,132ed57,Hồ Chí Minh,UNIDENTIFIED
3,F,ac6216,Hà Nội,U18
4,M,cb2f6f,Hà Nội,O55
...,...,...,...,...
66712,F,c91280,Hồ Chí Minh,35_44
66713,F,12fa13,Đà Nẵng,18_24
66714,F,1022ae,Hồ Chí Minh,39
66715,M,119452,Hồ Chí Minh,35_44


In [11]:
# Làm sạch cột age
age_values = df_d_user['age'].unique()
print("Danh sách các giá trị age:", age_values)
# Hàm phân loại theo nhóm tuổi
def classify_age_group(value):
    if isinstance(value, int):  # Kiểm tra nếu giá trị là số
        if 18 <= value <= 24:
            return '18_24'
        elif 25 <= value <= 34:
            return '25_34'
        elif 35 <= value <= 44:
            return '35_44'
        elif 45 <= value <= 54:
            return '45_54'
        elif value >= 55:
            return 'O55'
    elif isinstance(value, str):  # Handle string values
        if value == 'Hai Tư':  # Convert 'Hai Tư' to 24
            return '18_24'
        elif value == '28':  # Handle the string '28' as a number
            return '25_34'
    return value

# Tạo cột nhóm tuổi
df_d_user_add_column = df_d_user
df_d_user_add_column['age_group'] = df_d_user_add_column['age'].apply(classify_age_group)
df_d_user_clean = df_d_user_add_column[['gender','user_id','province_name','age_group']]
df_d_user_clean


Danh sách các giá trị age: ['O55' '25_34' 'UNIDENTIFIED' 'U18' '18_24' '35_44' '45_54' '28' 43 46 40
 35 44 38 47 45 37 41 36 39 42 'Hai Tư']


,gender,user_id,province_name,age_group
0,F,76843,Hà Nội,O55
1,UNIDENTIFIED,13346d5,Đà Nẵng,25_34
2,UNIDENTIFIED,132ed57,Hồ Chí Minh,UNIDENTIFIED
3,F,ac6216,Hà Nội,U18
4,M,cb2f6f,Hà Nội,O55
...,...,...,...,...
66712,F,c91280,Hồ Chí Minh,35_44
66713,F,12fa13,Đà Nẵng,18_24
66714,F,1022ae,Hồ Chí Minh,35_44
66715,M,119452,Hồ Chí Minh,35_44


In [12]:
# Config data types
d_user_data_types = {
    'gender': sqlalchemy.types.VARCHAR(length=255),
    'user_id': sqlalchemy.types.VARCHAR(length=25),
    'province_name': sqlalchemy.types.NVARCHAR(length=255),
    'age_group': sqlalchemy.types.VARCHAR(length=25)
}
# Load to database
df_d_user_clean.to_sql(name='d_user', con=engine, if_exists='append', index=False, dtype=d_user_data_types, schema = 'voucher')
print("Load data to table d_user done!")

Load data to table d_user done!
